In [1]:
import csv
import numpy as np

In [2]:
def calc_score_diff(play):
    """Calculate the score differential of the team with possession.

    Parameters:
    play(Dict): The play object.

    Returns:
    int: The score differential of the team with possession.
    """

    return int(play['posteam_score']) - int(play['defteam_score'])

def get_yrdln_int(play):
    """Given a play, get the line of scrimmage as an integer.

    Parameters:
    play(Dict): The play object.

    Returns:
    int: The yard line as an integer.
    """
    
    return int(play['yrdln'].split(' ')[-1])

def calc_seconds_since_halftime(play, year, is_postseason):
    """Calculate the number of seconds elapsed since halftime.

    Parameters:
    play(Dict): The play object.
    year(int): The year (season) of the game.
    is_postseason(bool): If the game is a postseason game.

    Returns:
    int: The number of seconds elapsed since halftime of that play.
    """
    
    if int(play['qtr']) <= 4:
        if play['game_seconds_remaining'] == '1e3':
            return 1800 - 1000
        return max(0, 1800-int(play['game_seconds_remaining']))
    
    # Check if game before 2017 or is postseason (when overtime rules change)
    if (year < 2017 or is_postseason):
        seconds_per_overtime = 900
    else:
        seconds_per_overtime = 600
        
    # Handle overtime
    if int(play['qtr']) == 5:
        return 1800 + seconds_per_overtime - int(play['game_seconds_remaining'])
    elif int(play['qtr']) == 6:
        return 3600 - int(play['game_seconds_remaining'])
    elif int(play['qtr']) == 7:
        return 4500 - int(play['game_seconds_remaining'])
    
    # Default
    return 0

def calc_field_pos_score(play):
    """Calculate the field position score for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The "field position score" for a given play,
           used to calculate the surrender index.
    """

    try:
        if '50' in play['yrdln']:
            return (1.1) ** 10.
        if play['posteam'] in play['yrdln']:
            return max(1., (1.1)**(get_yrdln_int(play) - 40))
        else:
            return (1.2)**(50 - get_yrdln_int(play)) * ((1.1)**(10))
    except BaseException:
        return 0.


def calc_yds_to_go_multiplier(play):
    """Calculate the yards to go multiplier for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The "yards to go multiplier" for a given play,
           used to calculate the surrender index.
    """

    if int(play['ydstogo']) >= 10:
        return 0.2
    elif int(play['ydstogo']) >= 7:
        return 0.4
    elif int(play['ydstogo']) >= 4:
        return 0.6
    elif int(play['ydstogo']) >= 2:
        return 0.8
    else:
        return 1.


def calc_score_multiplier(play):
    """Calculate the score multiplier for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The "score multiplier" for a given play,
           used to calculate the surrender index.
    """

    score_diff = calc_score_diff(play)

    if score_diff > 0:
        return 1.
    elif score_diff == 0:
        return 2.
    elif score_diff < -8.:
        return 3.
    else:
        return 4.


def calc_clock_multiplier(play, year, is_postseason):
    """Calculate the clock multiplier for a play.

    Parameters:
    play(Dict): The play object.
    year(int): The year (season) of the game.
    is_postseason(bool): If the game is a postseason game.

    Returns:
    float: The "clock multiplier" for a given play,
           used to calculate the surrender index.
    """

    if calc_score_diff(play) <= 0 and int(play['qtr']) > 2:
        seconds_since_halftime = calc_seconds_since_halftime(play, year, is_postseason)
        return ((seconds_since_halftime * 0.001) ** 3.) + 1.
    else:
        return 1.


def calc_surrender_index(play, year, is_postseason):
    """Calculate the surrender index for a play.

    Parameters:
    play(Dict): The play object.
    year(int): The year (season) of the game.
    is_postseason(bool): If the game is a postseason game.

    Returns:
    float: The surrender index for a given play.
    """

    return calc_field_pos_score(play) * calc_yds_to_go_multiplier(
        play) * calc_score_multiplier(play) * calc_clock_multiplier(play, year, is_postseason)

In [4]:
surrender_indices = []

for year in range(1999, 2024):
    with open('/Users/ashackelford/Downloads/pbp_data/play_by_play_' + str(year) + '.csv', 'r') as f:
        print(year)
        data = csv.DictReader(f)
        for play in data:
            if play['play_type'] == 'punt':
                surrender_indices.append(calc_surrender_index(play, year, play['season_type'] == 'POST'))

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [5]:
len(surrender_indices)

61996

In [6]:
np_surrender_indices = np.array(surrender_indices)

In [7]:
np.save('1999-2023_surrender_indices.npy', np_surrender_indices)